In [ ]:
import pandas as pd
import numpy as np
import glob
import time

import seaborn as sns
import matplotlib.pyplot as plt
from pandas_profiling import ProfileReport

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as md
import plotly.offline as py
import plotly.graph_objs as go
import plotly.tools as tls

from functools import reduce
from scipy import stats

from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC, LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVR
from sklearn.preprocessing import RobustScaler, MinMaxScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb

In [ ]:
data = pd.read_csv('data.csv')
df = pd.DataFrame.copy(data, deep=True)

In [ ]:
from numpy import mean
from numpy import std

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold

from sklearn.ensemble import StackingRegressor

from matplotlib import pyplot

def StackData():

    x = df[0:76588].drop(columns=['date', 'sup', 'energy', 'sup_diff', 'kwh'])
    y = df[0:76588].filter(['sup'])

    return x, y

# def GetStacking():

#     level0 = list()

#     level0.append(('svm', SVR()))
#     level0.append(('gb', GradientBoostingRegressor()))
#     level0.append(('rf', RandomForestRegressor()))

#     level1 = MLPRegressor()

#     model = StackingRegressor(estimators=level0, final_estimator=level1, cv=5)

#     return model

def GetModels():

    models = dict()

    models['svm'] = SVR()
    models['gb'] = GradientBoostingRegressor()
    models['rf'] = RandomForestRegressor()
    models['nn'] = MLPRegressor()

    return models

def EvaluateModel(model, x, y):

    cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=22)
    RMSE_score = cross_val_score(model, x,y , scoring='neg_root_mean_squared_error', cv=cv)

    return MAE_score, RMSE_score, RMLSE_score

x, y = StackData()

In [ ]:
models = GetModels()

results, names = list(), list()

for name, model in models.items():

    RMSE_score = EvaluateModel(model, x, y)

    results.append(RMSE_score)

    names.append(name)

    print('>RMSE: %s %.3f (%.3f)' % (name, mean(RMSE_score), std(RMSE_score)))

pyplot.boxplot(results, labels=names, showmeans=True)
pyplot.show()

In [ ]:
def PredictModels():

    predictions = list()

    X = df[88733:90928].drop(columns=['date', 'sup', 'energy', 'sup_diff', 'kwh'])
    Y = df[88733:90928].filter(['sup'])

    models = GetModels()

    for name, models in models.items():

        model.fit(x, y)
        results = model.predict(X)

        predictions.append(results)
        
    return predictions

predictions = PredictModels()

In [ ]:
DAT = pd.DataFrame()

Y = df[88733:90928].filter(['sup'])
Y.index = np.arange(0, len(Y))

DAT['svm'] = predictions[0]
DAT['gb'] = predictions[1]
DAT['rf'] = predictions[2]
DAT['nn'] = predictions[3]

sns.lineplot(data=DAT)